In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

%matplotlib inline

In [2]:
def crop_brain_contour(image, plot=False):
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image

In [3]:
def load_data(dir_list, image_size):
    # load all images in a directory
    X = []
    y = []
    image_width, image_height = image_size
    
    for directory in dir_list:
        for filename in listdir(directory):
            # load the image
            image = cv2.imread(directory + '\\' + filename)
            # crop the brain and ignore the unnecessary rest part of the image
            image = crop_brain_contour(image, plot=False)
            # resize image
            image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
            # normalize values
            image = image / 255.
            # convert image to numpy array and append it to X
            X.append(image)
            # append a value of 1 to the target array if the image
            # is in the folder named 'yes', otherwise append 0.
            if directory[-3:] == 'yes':
                y.append([1])
            else:
                y.append([0])
                
    X = np.array(X)
    y = np.array(y)
    
    X, y = shuffle(X, y)
    
    print(f'Number of examples is: {len(X)}')
    print(f'X shape is: {X.shape}')
    print(f'y shape is: {y.shape}')
    
    return X, y

In [4]:
import os
augmented_path =r'C:\Users\Muhammad Usama Abid\Desktop\uni\augmented_data'

# augmented data (yes and no) contains both the original and the new generated examples
augmented_yes = os.path.join(augmented_path , 'yes')
augmented_no = os.path.join(augmented_path , 'no')

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = load_data([augmented_yes, augmented_no], (IMG_WIDTH, IMG_HEIGHT))

Number of examples is: 2064
X shape is: (2064, 240, 240, 3)
y shape is: (2064, 1)


In [5]:
def split_data(X, y, test_size=0.2):
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3) 
# X_val, y_val = 310, X_train, y_train = 1445 , X_test, y_test = 310

In [7]:
def build_model(input_shape):
  
    # Define the input placeholder as a tensor with shape input_shape. 
    X_input = Input(input_shape) # shape=(?, 240, 240, 3)
    
    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((2, 2))(X_input) # shape=(?, 244, 244, 3)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X) # shape=(?, 238, 238, 32)
    
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool0')(X) # shape=(?, 59, 59, 32) 
    
    # MAXPOOL
    X = MaxPooling2D((4, 4), name='max_pool1')(X) # shape=(?, 14, 14, 32)
    
    # FLATTEN X 
    X = Flatten()(X) # shape=(?, 6272)
    # FULLYCONNECTED
    X = Dense(1, activation='sigmoid', name='fc')(X) # shape=(?, 1)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='BrainDetectionModel')
    
    return model

In [8]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [9]:
model = build_model(IMG_SHAPE)

In [10]:
model.summary()

Model: "BrainDetectionModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 244, 244, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 238, 238, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 238, 238, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 238, 238, 32)      0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 59, 59, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 14, 14, 32)

In [11]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [12]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [13]:
def compute_f1_score(y_true, prob):
    # convert the vector of probabilities to a target vector
    y_pred = np.where(prob > 0.5, 1, 0)
    
    score = f1_score(y_true, y_pred)
    
    return score

In [ ]:
start_time = time.time()

model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

end_time = time.time()
execution_time = (end_time - start_time)
print(f"Elapsed time: {hms_string(execution_time)}")

In [ ]:
def plot_metrics(history):
    
    train_loss = history['loss']
    val_loss = history['val_loss']
    train_acc = history['accuracy']
    val_acc = history['val_accuracy']
    
    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
    
    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
history = model.history.history
print(history)

In [ ]:
for key in history.keys():
    print(key)

In [ ]:
plot_metrics(history)

In [14]:
loss, acc = model.evaluate(x=X_test, y=y_test)

10/10 [==============================] - 5s 509ms/step - loss: 0.2509 - accuracy: 0.5226


In [ ]:
print (f"Test Loss = {loss}")
print (f"Test Accuracy = {acc}")

In [ ]:
model.save('brain-tumor-detection-CNN.h5')

In [15]:
from keras.models import load_model
new_model = load_model('brain-tumor-detection-CNN.h5')
new_model.summary()

Model: "BrainDetectionModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 244, 244, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 238, 238, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 238, 238, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 238, 238, 32)      0         
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 59, 59, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 14, 14, 32)

In [17]:
pred = new_model.predict(X_test,batch_size=32, verbose = 0)
#pred
#np.argmax(pred)
for i in pred:
    print(i)
    #n= int(i)
    #print(np.argmax(pred[i]))

[0.49410746]
[0.9980666]
[0.01362911]
[0.18657252]
[0.00275993]
[0.9832089]
[0.9897305]
[0.9988812]
[0.999759]
[0.00023142]
[3.2726828e-05]
[0.99550986]
[5.3533437e-05]
[0.83686924]
[0.05899534]
[0.9970455]
[0.9369131]
[0.00487792]
[0.9923124]
[0.00202364]
[0.0129441]
[0.99997854]
[0.9913244]
[0.00043449]
[0.81557]
[0.00052491]
[0.8574614]
[0.00121778]
[0.23388007]
[0.11970419]
[0.9957808]
[0.5771359]
[0.99938667]
[0.43254554]
[0.37187314]
[0.07996193]
[0.00269815]
[0.00239012]
[0.97233796]
[0.97930884]
[0.2905047]
[0.00635245]
[0.4772828]
[0.00965342]
[0.9997168]
[0.36777943]
[0.991458]
[0.00010639]
[0.02370632]
[0.9999007]
[0.78968513]
[0.06134874]
[0.00500569]
[0.31351072]
[0.04144242]
[0.01903579]
[0.9275755]
[0.01870328]
[0.11820567]
[0.2891546]
[0.95400286]
[0.03833172]
[0.01548702]
[0.9992626]
[0.02451098]
[0.00338313]
[0.14234358]
[0.8273866]
[0.99216354]
[0.5588769]
[0.37242895]
[0.00221393]
[0.11534318]
[0.0410493]
[0.935501]
[0.97772384]
[0.01519361]
[0.9806727]
[0.04397351]

In [19]:
image = cv2.imread("aug_Y65_0_7028.jpg")
#image.shape #(630,630,3)
image_size = (240,240)
image_width, image_height = image_size
image = crop_brain_contour(image, plot=False)
image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
image = image / 255.
image = np.array(image).reshape(1,240,240,3)
prediction = new_model.predict(image ,batch_size=32, verbose = 0)


In [22]:
print(prediction)
#catig = ["yes", "no"]
#print(catig[np.argmax(predictions[0])])
#print(np.argmax(prediction[0]))

[[0.9853593]]


In [ ]:
json_string = model.to_json()
json_string

In [ ]:
class_names = ['yes', 'no']
#predictions = model.predict([test_images])[0]
predicted_label = class_names[np.argmax(prediction)]
# Compare the predictions

print("Predictions : ",predicted_label)
#print("Actual : ", class_names[y_test[0]])

In [ ]:
catig = ["yes", "no"]
#rounded_predictions = new_model(X_test ,batch_size= 32, verbose = 0)
predictions = model.predict(X ,batch_size= 10, verbose = 0)


In [ ]:
y_true = np.array(['yes', 'no'])
y_pred = np.array([1,0])
compute_f1_score(y_val, y_test)